In [14]:
import pkg_resources
pkg_resources.require("qutip==4.7.1")
import qutip as qt

In [15]:

import sys 

import SQcircuit_extensions as sq_ext
#import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt
from matplotlib.gridspec import GridSpec


%matplotlib ipympl
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
importlib.reload(sq_ext)
#importlib.reload(sq)
np.set_printoptions(linewidth=300, formatter={'float': '{:.12f}'.format})

AttributeError: module 'qutip.settings' has no attribute 'core'

In [3]:
fF  = 1e-15
nH  = 1e-9
GHz = 1e9
h   = 6.626e-34
e0  = 1.602e-19
phi_0 = h / (2 * e0)

In [4]:
opt_dir = 'opt_results'

In [5]:
fq2_set_vec = np.array([3.62  , 3.6245, 3.629 , 3.6335, 3.638 , 3.6425, 3.647 , 3.6515,
       3.656 , 3.6605, 3.665 , 3.6695, 3.674 , 3.6785, 3.683 , 3.6875,
       3.692 , 3.6965, 3.701 , 3.7055, 3.71  ])
f2_of_max_g = 3.6587
x0_array_opt = np.array([0.00026772, 0.00030737, 0.00034445, 0.00042214, 0.00055151,
       0.00069153, 0.00096878, 0.        , 0.        , 0.        ,
       0.0014975 , 0.0010779 , 0.00085284, 0.00067319, 0.00059366,
       0.00048784, 0.00044596, 0.00041887, 0.00035964, 0.00038036,
       0.00031654])

# Qubit 1 - Qubit 3, no resonator, vs phi_ext_2

In [6]:
CF_1, LF_1, EJ_1, _, _      = np.load(opt_dir +  '/qubit_1' + '.npz')['parameters_opt']
C_int_11, CR_1, LR_1, _, _  = np.load(opt_dir +  '/resonator_1' + '.npz')['parameters_opt']
CF_2, LF_2, EJ_2, _, _      = np.load(opt_dir +  '/qubit_2' + '.npz')['parameters_opt']
CR_2, LR_2, Δ_2, _, _       = np.load(opt_dir +  '/resonator_2' + '.npz')['parameters_opt']
CF_3, LF_3, EJ_3, _, _      = np.load(opt_dir +  '/qubit_3' + '.npz')['parameters_opt']
C_int_33, CR_3, LR_3, _, _  = np.load(opt_dir +  '/resonator_3' + '.npz')['parameters_opt']
C_int_12, _, _, _           = np.load(opt_dir +  '/qubit_1_qubit_2' + '.npz')['parameters_opt']
C_int_12, C_int_23, C_int_13, _, _, _, _, _, _, _, _, _= np.load(opt_dir +  '/qubit_1_qubit_2_qubit_3' + '.npz')['parameters_opt'] 
# C_int_23 = C_int_12
C_int_13 = 550000.000
# C_int_13 = 1e10

In [7]:
C_diag       = [CF_1, CR_1, CF_2, CR_2, CF_3, CR_3  ]
C_int_diag   = [C_int_11, _, C_int_33               ]   
C_int        = [C_int_12, C_int_23, C_int_13        ]
C_inv = sq_ext.C_mat_from_C_int(C_diag, C_int_diag, C_int, only_qubit_modes=True)

In [8]:
nmax_f = 10
nmax_r =5
# φ_ext_2_values = np.linspace(0.5165,0.519,100)
n_eig_circuits=3
n_eig_qubit=4
extra_eigs = 10

In [9]:
phi_ext_2_values = np.concatenate(([0.5], np.linspace(0.516, 0.52,101), [0.52668]))

In [10]:
p1              = np.zeros([len(φ_ext_2_values), 4, 4])
p2              = np.zeros([len(φ_ext_2_values), 4, 4])
SWT             = np.zeros([len(φ_ext_2_values), 4, 4])
Qs              = np.zeros([len(φ_ext_2_values), 4, 4], dtype='complex')
g_13_analytic   = np.zeros([len(φ_ext_2_values)])
Δfq2            = np.zeros([len(φ_ext_2_values)])

fluxonium_1 = sq_ext.sq_fluxonium(C_F_eff=C_inv[0, 0] ** -1, L_F_eff = LF_1, EJ=EJ_1, nmax_f=nmax_f)
fluxonium_3 = sq_ext.sq_fluxonium(C_F_eff=C_inv[2, 2] ** -1, L_F_eff = LF_3, EJ=EJ_3, nmax_f=nmax_f)

φ_ext_3_resonance = sq_ext.find_resonance(fluxonium_1.hamiltonian(), fluxonium_3)
loop_3 = fluxonium_3.loops[0]
loop_3.set_flux(φ_ext_3_resonance)

In [11]:
for i, φ_ext_2 in enumerate(φ_ext_2_values):
    
    print(i)
    
    fluxonium_2 = sq_ext.sq_fluxonium(C_F_eff=C_inv[1, 1] ** -1, L_F_eff = LF_2, EJ=EJ_2, nmax_f=nmax_f, φ_ext=φ_ext_2)
    if i==0:
        ω_1 = sq_ext.diag(fluxonium_1.hamiltonian(),2,remove_ground=True)[0][1]
        ω_3 = sq_ext.diag(fluxonium_3.hamiltonian(),2,remove_ground=True)[0][1]
    ω_2 = sq_ext.diag(fluxonium_2.hamiltonian(),2,remove_ground=True)[0][1]
    Δfq2[i] = (ω_1 - ω_2)*1e3
    print(ω_1,ω_2, ω_3)
    
    circuits = [fluxonium_1, fluxonium_2, fluxonium_3]
    H_circuits = [circuit.hamiltonian() for circuit in circuits]
    E_circuits = [sq_ext.diag(H, n_eig_circuits, solver='numpy')[0] for H in H_circuits]
    ψ_circuits = [sq_ext.diag(H, n_eig_circuits, solver='numpy')[1] for H in H_circuits]
    
    H_0, H   = sq_ext.hamiltonian_fluxonium_C_fluxonium_C_fluxonium(C_inv, circuits, nmax_f=nmax_f, return_H_0=True)
    V = H-H_0

    ψ_00 = qt.tensor([qt.Qobj(ψ_circuits[0][:,0]), qt.Qobj(ψ_circuits[1][:,0]), qt.Qobj(ψ_circuits[2][:,0])])
    ψ_10 = qt.tensor([qt.Qobj(ψ_circuits[0][:,1]), qt.Qobj(ψ_circuits[1][:,0]), qt.Qobj(ψ_circuits[2][:,0])])
    ψ_01 = qt.tensor([qt.Qobj(ψ_circuits[0][:,0]), qt.Qobj(ψ_circuits[1][:,0]), qt.Qobj(ψ_circuits[2][:,1])])
    ψ_11 = qt.tensor([qt.Qobj(ψ_circuits[0][:,1]), qt.Qobj(ψ_circuits[1][:,0]), qt.Qobj(ψ_circuits[2][:,1])])

    ψ_0 = [ψ_00,ψ_01,ψ_10,ψ_11]
    
    ψ_010 = qt.tensor([qt.Qobj(ψ_circuits[0][:,0]), qt.Qobj(ψ_circuits[1][:,1]), qt.Qobj(ψ_circuits[2][:,0])])
    
    ψ_110 = qt.tensor([qt.Qobj(ψ_circuits[0][:,1]), qt.Qobj(ψ_circuits[1][:,1]), qt.Qobj(ψ_circuits[2][:,0])])
    ψ_011 = qt.tensor([qt.Qobj(ψ_circuits[0][:,0]), qt.Qobj(ψ_circuits[1][:,1]), qt.Qobj(ψ_circuits[2][:,1])])
    ψ_111 = qt.tensor([qt.Qobj(ψ_circuits[0][:,1]), qt.Qobj(ψ_circuits[1][:,1]), qt.Qobj(ψ_circuits[2][:,1])])
    
    ψ_200 = qt.tensor([qt.Qobj(ψ_circuits[0][:,2]), qt.Qobj(ψ_circuits[1][:,0]), qt.Qobj(ψ_circuits[2][:,0])])
    ψ_020 = qt.tensor([qt.Qobj(ψ_circuits[0][:,0]), qt.Qobj(ψ_circuits[1][:,2]), qt.Qobj(ψ_circuits[2][:,0])])
    ψ_002 = qt.tensor([qt.Qobj(ψ_circuits[0][:,0]), qt.Qobj(ψ_circuits[1][:,0]), qt.Qobj(ψ_circuits[2][:,2])])
    
    ψ_0_high = [ψ_010, ψ_110, ψ_011, ψ_111, ψ_200, ψ_020, ψ_002]

    E_0_ψ_0  = [ np.real((ψ_0_i.dag() * H_0 * ψ_0_i).data[0,0])/2/np.pi/GHz for ψ_0_i in ψ_0]
    E_0_ψ_0_high  = [ np.real((ψ_0_i.dag() * H_0 * ψ_0_i).data[0,0])/2/np.pi/GHz for ψ_0_i in ψ_0_high]
    E_0  = sq_ext.diag(H_0 , n_eig=len(ψ_0)+extra_eigs, out='GHz', solver='scipy')[0]
    
    E_000 = E_0_ψ_0[0]
    
    E_0_ψ_0         -= E_000
    E_0_ψ_0_high    -= E_000
    E_0             -= E_000
    print(E_0_ψ_0, E_0)
    subspace_indices = sq_ext.find_close_indices(E_0_ψ_0, E_0)
    print(subspace_indices)
    print('/n')

    high_subspace_indices = sq_ext.find_close_indices(E_0_ψ_0_high, E_0)
    valid_high_indices = np.where(np.array(high_subspace_indices) > max(subspace_indices))[0]
    high_subspace_indices = [high_subspace_indices[valid_high_index] for valid_high_index in valid_high_indices]
    
    print(high_subspace_indices)
    
    # ψ_0_high        = [ψ_0_high[index]      for index in valid_high_indices]
    # E_0_ψ_0_high    = [E_0_ψ_0_high[index]  for index in valid_high_indices]

    E, ψ = sq_ext.diag(H  , n_eig=len(ψ_0)+extra_eigs, out='GHz', solver='Qutip', qObj=True)
    ψ = ψ[subspace_indices]
    E = E[subspace_indices]
    
    # print('Punto de control 1')
    H_eff_p1  = sq_ext.H_eff_p1_large (ψ_0, H, real=True, remove_ground=True)
    H_eff_p2  = H_eff_p1 + sq_ext.H_eff_p2_large(ψ_0, ψ_0_high, E_0_ψ_0, E_0_ψ_0_high, V, remove_ground=True)
    H_eff_SWT, Qs[i,:] = sq_ext.H_eff_SWT_large(ψ_0, ψ, E, remove_ground=True,return_Q=True)
    
    p1 [i,:] = sq_ext.decomposition_in_pauli_4x4(H_eff_p1  ,  print_pretty=False)
    p2 [i,:] = sq_ext.decomposition_in_pauli_4x4(H_eff_p2  ,  print_pretty=False)
    SWT[i,:] = sq_ext.decomposition_in_pauli_4x4(H_eff_SWT , print_pretty=False)
    
    #Analytics:
    fluxonium_1.diag(2)
    fluxonium_2.diag(2)
    fluxonium_3.diag(2)
    
    Q_1 = np.abs(fluxonium_1.charge_op(0, basis='eig')[0,1])
    Q_2 = np.abs(fluxonium_2.charge_op(0, basis='eig')[0,1])
    Q_3 = np.abs(fluxonium_3.charge_op(0, basis='eig')[0,1])
    
    ω_1 = sq_ext.diag(fluxonium_1.hamiltonian(),2, remove_ground=True)[0][1]
    ω_2 = sq_ext.diag(fluxonium_2.hamiltonian(),2, remove_ground=True)[0][1]
    ω_3 = sq_ext.diag(fluxonium_3.hamiltonian(),2, remove_ground=True)[0][1]
    
    Δ_12 = (ω_1 - ω_2) # maybe change sign
    Δ_23 = (ω_3 - ω_2) 
    Σ_12 = ω_1 + ω_2
    Σ_23 = ω_3 + ω_2

    g_12 = C_inv[0, 1] * fF**-1 * Q_1 * Q_2 /2/np.pi/GHz
    g_23 = C_inv[1, 2] * fF**-1 * Q_2 * Q_3 /2/np.pi/GHz
    g_13 = C_inv[0, 2] * fF**-1 * Q_1 * Q_3 /2/np.pi/GHz

    g_eff = g_12*g_23*(1/Δ_12 + 1/Δ_23 - 1/Σ_12 - 1/Σ_23)/2 + g_13

    g_13_analytic[i] = g_eff
    

0
3.6901515218282483 3.437286972836433 3.69018458721844
[0.000000000000 3.690184587218 3.690151521828 7.380336109047] [0.000000000000 3.437286972837 3.690151521829 3.690184587219 7.127438494665 7.127471560055 7.380336109047 7.813782432210 8.107053050788 8.568490758873 10.817623081883 11.251069405047 11.503933954039 11.544340023624]
[0, 3, 2, 6]
/n
[10, 8, 9, 7]


C:\Users\gabri\PycharmProjects\tunable-capacitive-coupling-flux-qubits\Self contained for experimentalists\SQcircuit_extensions.py:1811: ComplexWarning: Casting complex values to real discards the imaginary part
  P[i, j] = 0.25 * (np.dot(S.T.conjugate(), A)).trace() # P[i,j]=(1/4)tr(S_ij^t*A)


1
3.690151521828252 3.6317257156232703 3.6901845872184333
[0.000000000000 3.690184587218 3.690151521828 7.380336109047] [0.000000000000 3.631725715624 3.690151521828 3.690184587219 7.321877237452 7.321910302842 7.380336109047 7.813782432210 8.107053050788 8.720254393719 11.012061824670 11.445508147834 11.503933954039 11.738778766411]
[0, 3, 2, 6]
/n
[10, 8, 9, 7]
2
3.6901515218282634 3.632659253576932 3.69018458721845
[0.000000000000 3.690184587218 3.690151521828 7.380336109047] [0.000000000000 3.632659253577 3.690151521829 3.690184587219 7.322810775405 7.322843840796 7.380336109047 7.813782432210 8.107053050788 8.720996299483 11.012995362624 11.446441685787 11.503933954038 11.739712304365]
[0, 3, 2, 6]
/n
[10, 8, 9, 7]
3
3.6901515218282315 3.6335947462475886 3.690184587218436
[0.000000000000 3.690184587218 3.690151521828 7.380336109047] [0.000000000000 3.633594746248 3.690151521829 3.690184587219 7.323746268076 7.323779333466 7.380336109047 7.813782432210 8.107053050788 8.721739889545

In [ ]:

fig = plt.figure(dpi=150, figsize=np.array([8, 6])*0.6)
gs = GridSpec(1, 2, width_ratios=[1, 10])  # Width ratio of 1:10

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharey=ax1)

linestyles = ['-', '--', '-' ]
labels = [r'$|h_X|$', r'$|h_Y|$', r'$|h_Z|$']
colors = figs.generate_colors_from_colormap(10, 'tab10')
Pauli_list = [p1, p2, SWT]

left_indices_plot  =  np.where(Δfq2 <= 0.5 * np.max((fq2_set_vec[:7]  - f2_of_max_g) * 1e3)) 
right_indices_plot = np.where(Δfq2 >= 0.5 * np.min((fq2_set_vec[-11:] - f2_of_max_g) * 1e3))

# Plot data on the first subplot with the range -215 to -200 (broken axis on the left)

detuned_Δfq2 = -252
detuned_g13 = 0.04917

detuned_index = np.where(np.abs(Δfq2 - detuned_Δfq2) == np.abs(Δfq2- detuned_Δfq2).min())[0][0]

ax1.plot(detuned_Δfq2, detuned_g13, marker='o', ls='', color='m', ms=5, label='pt used for exp fit, extract with spectros.')


ax1.set_xlim(-255, -250)
ax1.spines['right'].set_visible(False)
ax1.yaxis.tick_left()

ax2.spines['left'].set_visible(False)
ax2.yaxis.tick_right()

# Add diagonal lines to indicate the break
d = .015  # size of the diagonal lines
kwargs = dict(transform=ax2.transAxes, color='k', clip_on=False)
ax2.plot((-d, +d), (-d, +d), **kwargs)  # top-left diagonal
ax2.plot((-d, +d), (1-d, 1+d), **kwargs)  # bottom-left diagonal


ax2.plot((fq2_set_vec[:7]-f2_of_max_g)*1e3, x0_array_opt[:7]*1e3, color='m', marker='o', ls='', ms=5,label='spectrum')
ax2.plot((fq2_set_vec[-11:]-f2_of_max_g)*1e3, x0_array_opt[-11:]*1e3, color='m', marker='o', ls='', ms=5)
ax2.plot(0,3.16 / np.sqrt(2), color='m', marker='x', ls='', ms=5, label='Ramsey')
ax2.plot([], 'k', markersize=3, label='2° SWT')

for i, Pauli in enumerate(Pauli_list):
    if i == 0 or i==1:
        continue
    σxσx = np.abs(Pauli[:, 1, 1])
    σyσy = np.abs(Pauli[:, 2, 2])
    σzσz = np.abs(Pauli[:, 3, 3])
    
    ax2.plot(Δfq2 , σyσy  * 1e3, color=colors[1], linestyle=linestyles[i], label=labels[1])
    ax2.plot(Δfq2 , σxσx  * 1e3, color=colors[0], linestyle=linestyles[i], label=labels[0])
    ax2.plot(Δfq2 , σzσz  * 1e3, color=colors[2], linestyle=linestyles[i], label=labels[2])

    if i ==2:
        ax2.legend(fontsize=8)
        
    if i==2:
        ax1.plot(Δfq2[detuned_index], np.abs(σyσy[detuned_index] * 1e3), '.:', color=colors[1])
        ax1.plot(Δfq2[detuned_index], np.abs(g_13_analytic[detuned_index] * 1e3), '.k', markersize=3)
        
ax2.plot(Δfq2, np.abs(g_13_analytic * 1e3), 'k', markersize=3)

ax1.set_yscale('log')
ax2.set_xlabel(r'$\Delta(f_{q2})$ (MHz)')
ax1.set_ylabel(r'$g_{13}$ (MHz)')

ax2.set_ylim([1e-3, 4e0])
ax2.set_xlim([-45, 60])
fig.tight_layout()

# Export or show the figure
# fig.show()
